In [1]:
import pandas as pd
from preprocessing import get_models_and_soups_df
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
plt.style.use("style.mplstyle")

In [2]:
soups, models = get_models_and_soups_df()
soups['soup_gain'] = soups['clean_accuracy'] - soups[['clean_accuracy_a', 'clean_accuracy_b']].max(axis=1)

In [3]:
soups.columns

Index(['key_a', 'key_b', 'epoch_a', 'variant_a', 'epoch_b', 'variant_b',
       'shared_epochs', 'clean_accuracy', 'clean_loss', 'corrupted_accuracy',
       'corrupted_loss', 'clean_accuracy_permuted', 'clean_loss_permuted',
       'l2_distance', 'cosine_similarity', 'cka_logits', 'mse_logits',
       'kl_logits', 'cka_features', 'clean_loss_a', 'clean_accuracy_a',
       'clean_loss_b', 'clean_accuracy_b', 'corrupted_loss_a',
       'corrupted_loss_b', 'soup_gain', 'permutated_gain', 'corrupted_gain'],
      dtype='object')

In [4]:
import numpy as np
import pandas as pd

np.random.seed(42)

B = 1000
alpha = 0.05
targets = [200, 250]

rows = []

for se in targets:
    df = soups[soups['shared_epochs'] == se].dropna(
        subset=['soup_gain', 'corrupted_gain']
    )

    gains = df['soup_gain'].to_numpy()
    corr_gains = df['corrupted_gain'].to_numpy()
    n = gains.size
    if n == 0:
        continue

    # point estimates
    p_pos = (gains > 0).mean()
    mean_pos = gains[gains > 0].mean() if np.any(gains > 0) else np.nan
    mean_corr_pos = corr_gains[gains > 0].mean() if np.any(gains > 0) else np.nan

    # bootstrap
    boot_p = np.empty(B)
    boot_m = np.empty(B)
    boot_mc = np.empty(B)

    for b in range(B):
        idx = np.random.randint(0, n, size=n)
        g = gains[idx]
        cg = corr_gains[idx]

        boot_p[b] = (g > 0).mean()

        pos = g > 0
        boot_m[b]  = g[pos].mean()  if np.any(pos) else np.nan
        boot_mc[b] = cg[pos].mean() if np.any(pos) else np.nan

    rows.append({
        'shared_epochs': se,

        'P(gain>0)': p_pos,
        'P_lo': np.quantile(boot_p, alpha/2),
        'P_hi': np.quantile(boot_p, 1 - alpha/2),

        'E[gain | gain>0]': mean_pos,
        'E_lo': np.nanquantile(boot_m, alpha/2),
        'E_hi': np.nanquantile(boot_m, 1 - alpha/2),

        'E[corr gain | gain>0]': mean_corr_pos,
        'C_lo': np.nanquantile(boot_mc, alpha/2),
        'C_hi': np.nanquantile(boot_mc, 1 - alpha/2),

        'n': n
    })

out_df = pd.DataFrame(rows).sort_values('shared_epochs')
out_df


,shared_epochs,P(gain>0),P_lo,P_hi,E[gain | gain>0],E_lo,E_hi,E[corr gain | gain>0],C_lo,C_hi,n
0,200,0.436242,0.355537,0.510235,0.603231,0.563415,0.642109,0.015035,0.009371,0.020704,149
1,250,0.511628,0.406977,0.604651,0.159318,0.122495,0.197319,0.000316,-0.002869,0.003028,86


In [8]:
out_df['P_pm'] = 0.5 * (out_df['P_hi'] - out_df['P_lo'])
out_df['E_pm'] = 0.5 * (out_df['E_hi'] - out_df['E_lo'])
out_df['C_pm'] = 0.5 * (out_df['C_hi'] - out_df['C_lo'])

In [11]:
fmt = lambda m, lo, hi: f'{m:.3f} [{lo:.3f}, {hi:.3f}]'

pretty = pd.DataFrame({
    'shared_epochs': out_df['shared_epochs'],
    'P(gain > 0)': [
        fmt(m, lo, hi) for m, lo, hi in
        zip(out_df['P(gain>0)'], out_df['P_lo'], out_df['P_hi'])
    ],
    'E[gain | gain > 0]': [
        fmt(m, lo, hi) for m, lo, hi in
        zip(out_df['E[gain | gain>0]'], out_df['E_lo'], out_df['E_hi'])
    ],
    'E[corr gain | gain > 0]': [
        fmt(m, lo, hi) for m, lo, hi in
        zip(out_df['E[corr gain | gain>0]'], out_df['C_lo'], out_df['C_hi'])
    ],
    'n': out_df['n']
})

pretty


,shared_epochs,P(gain > 0),E[gain | gain > 0],E[corr gain | gain > 0],n
0,200,"0.436 [0.356, 0.510]","0.603 [0.563, 0.642]","0.015 [0.009, 0.021]",149
1,250,"0.512 [0.407, 0.605]","0.159 [0.122, 0.197]","0.000 [-0.003, 0.003]",86


In [10]:
fmt_pm = lambda m, pm: f'{m:.3f} ± {pm:.3f}'

pretty_pm = pd.DataFrame({
    'shared_epochs': out_df['shared_epochs'],
    'P(gain > 0)': [
        fmt_pm(m, pm) for m, pm in zip(out_df['P(gain>0)'], out_df['P_pm'])
    ],
    'E[gain | gain > 0]': [
        fmt_pm(m, pm) for m, pm in zip(out_df['E[gain | gain>0]'], out_df['E_pm'])
    ],
    'E[corr gain | gain > 0]': [
        fmt_pm(m, pm) for m, pm in zip(out_df['E[corr gain | gain>0]'], out_df['C_pm'])
    ],
    'n': out_df['n']
})

pretty_pm

,shared_epochs,P(gain > 0),E[gain | gain > 0],E[corr gain | gain > 0],n
0,200,0.436 ± 0.077,0.603 ± 0.039,0.015 ± 0.006,149
1,250,0.512 ± 0.099,0.159 ± 0.037,0.000 ± 0.003,86


Note the above CIs are computed by bootstrapping a 95% to obtain a width, then centering this width around the observed value. However, these are nearly the same as can be seen by comparing the two tables above.